# <center>Machine Learning Project</center>

** **
## <center>*03.5 - Random Forest*</center>

** **

Description of the notebook

Simple Model with evaluation


The members of the `team` are:
- Ana Farinha - 20211514
- Francisco Capontes - 20211692
- Sofia Gomes - 20240848
- Rui Lourenço - 2021639


## <span style="color:salmon"> Table of Contents </span>

<a class="anchor" id="top"></a>

1. [Importing Libraries & Data](#one-bullet) 

2. [Prepare Dataset](#two-bullet)

   2.1 [Separate Target](#two-one-bullet) 

   2.2 [Train Validation Split](#two-two-bullet)
   
   2.3 [Impute Missing Values](#two-three-bullet) 
   
   2.4 [Scaling Data](#two-four-bullet) <br><br>

3. [Model Training](#three-bullet)

4. [Model Evaluation](#four-bullet) 

   4.1 [Training Evaluation](#four-one-bullet) 
   
   4.2 [Validation Evaluation](#four-two-bullet) <br><br>

5. [Test Prediction](#five-bullet) 

<a class="anchor" id="one-bullet"> 
    
# <span style="color:salmon"> 1. Importing Libraries & Data </span> 
</a>    
</div>

<a href="#top">Top &#129033;</a>

In [78]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

from utils import *

In [79]:
# Import  for training and validation
df = pd.read_csv('./preprocessed_data/train_data.csv', index_col = 'Claim Identifier')
df

,Accident Date,Age at Injury,Average Weekly Wage,Birth Year,IME-4 Count,Number of Dependents,Known Accident Date,Known Assembly Date,Known C-2 Date,Known C-3 Date,...,Zip_Code_Hash_13,Zip_Code_Hash_14,Zip_Code_Hash_15,Zip_Code_Hash_16,Zip_Code_Hash_17,Zip_Code_Hash_18,Zip_Code_Hash_19,Claim Injury Type Encoded,Accident_Season_Sin,Accident_Season_Cos
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
5393875,2019-12-30,31.0,NaN,1988.0,0.0,1.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.000000e+00,1.000000e+00
5393091,2019-08-30,46.0,1745.93,1973.0,4.0,4.0,1,1,1,1,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,3,1.224647e-16,-1.000000e+00
5393889,2019-12-06,40.0,1434.80,1979.0,0.0,6.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,3,-1.000000e+00,-1.836970e-16
5393887,2019-12-30,61.0,NaN,1958.0,0.0,1.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.000000e+00,1.000000e+00
5393863,2019-12-26,67.0,NaN,1952.0,0.0,5.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.000000e+00,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6165265,2022-12-14,35.0,NaN,1987.0,0.0,4.0,1,1,1,0,...,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,1,-1.000000e+00,-1.836970e-16
6165285,2022-12-15,33.0,NaN,1989.0,0.0,6.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,-1.000000e+00,-1.836970e-16
6165506,2022-12-13,61.0,991.08,1961.0,1.0,3.0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,3,-1.000000e+00,-1.836970e-16


In [80]:
# Import dataset
test_df = pd.read_csv('./preprocessed_data/test_data.csv', index_col = 'Claim Identifier')
test_df

KeyboardInterrupt: 

<a class="anchor" id="two-bullet"> 
    
# <span style="color:salmon"> 2. Prepare Dataset </span> 
</a>    
</div>
<a href="#top">Top &#129033;</a>

<a class="anchor" id="two-one-bullet"> 
    
## <span style="color:salmon"> 2.1 Separate Target </span> 
</a>    
</div>

In [ ]:
# Define features and target
X = df.drop('Claim Injury Type Encoded', axis=1)
y = df['Claim Injury Type Encoded']

<a class="anchor" id="two-twpo-bullet"> 
    
## <span style="color:salmon"> 2.2 Train Validation Split </span> 
</a>    
</div>

In [ ]:
# Split the data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
X_test = test_df.copy()

<a class="anchor" id="two-three-bullet"> 
    
## <span style="color:salmon"> 2.3 Impute Missing Values </span> 
</a>    
</div>

*Check missing values in each dataset*

In [ ]:
X_train.isna().sum().sort_values()[X_train.isna().sum() != 0]

Age at Injury            1758
Birth Year               1758
Average Weekly Wage    291230
dtype: int64

In [ ]:
X_val.isna().sum().sort_values()[X_val.isna().sum() != 0]

Age at Injury            428
Birth Year               428
Average Weekly Wage    72869
dtype: int64

In [ ]:
X_test.isna().sum().sort_values()[X_test.isna().sum() != 0]

Age at Injury             982
Birth Year                982
Average Weekly Wage    335753
dtype: int64

*Input `Average Weekly Wage` and variables created from it*

In [ ]:
median_avg_week_wage = X_train['Average Weekly Wage'].median()

X_train['Average Weekly Wage'] = X_train['Average Weekly Wage'].fillna(median_avg_week_wage)
X_val['Average Weekly Wage'] = X_val['Average Weekly Wage'].fillna(median_avg_week_wage)
X_test['Average Weekly Wage'] = X_test['Average Weekly Wage'].fillna(median_avg_week_wage)

In [ ]:
financial_impact(X_train)
financial_impact(X_val)
financial_impact(X_test)

*Input `Age at Injury` and variables that can be calculated with it*

In [ ]:
median_age_at_injury = X_train['Age at Injury'].median()

X_train['Age at Injury'] = X_train['Age at Injury'].fillna(median_age_at_injury)
X_val['Age at Injury'] = X_val['Age at Injury'].fillna(median_age_at_injury)
X_test['Age at Injury'] = X_test['Age at Injury'].fillna(median_age_at_injury)

In [ ]:
def calculate_birth_year(df):
    # Ensure the correct format of 'Birth Year'
    df['Accident Date'] = pd.to_datetime(df['Accident Date'], errors='coerce')

    # Filter the rows where 'Birth Year' is NaN, but 'Age at Injury' and 'Accident Date' are not NaN
    condition = df['Birth Year'].isna() & df['Age at Injury'].notna() & df['Accident Date'].notna()

    # Replace missing 'Birth Year' with the difference between 'Accident Date' year and 'Age at Injury'
    df.loc[condition, 'Birth Year'] = df.loc[condition, 'Accident Date'].dt.year - df.loc[condition, 'Age at Injury']

In [ ]:
calculate_birth_year(X_train)
calculate_birth_year(X_val)
calculate_birth_year(X_test)

*Drop Dates*

In [ ]:
X_train.drop('Accident Date', axis=1, inplace=True)
X_val.drop('Accident Date', axis=1, inplace=True)
X_test.drop('Accident Date', axis=1, inplace=True)

<a class="anchor" id="two-four-bullet"> 
    
## <span style="color:salmon"> 2.4 Scaling Data </span> 
</a>    
</div>

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

<a class="anchor" id="three-bullet"> 
    
# <span style="color:salmon"> 3. Model Training </span> 
</a>    
</div>

<a href="#top">Top &#129033;</a>

In [ ]:
class_mapping = {
    0:'1. CANCELLED', 
    1:'2. NON-COMP',
    2:'3. MED ONLY', 
    3:'4. TEMPORARY',
    4:'5. PPD SCH LOSS', 
    5:'6. PPD NSL', 
    6:'7. PTD', 
    7:'8. DEATH'
}

# Use the values from class_mapping as the target names
target_names = list(class_mapping.values())

In [ ]:
# Initialize and train the DecisionTreeClassifier
model = RandomForestClassifier(min_samples_split=50, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(min_samples_split=50, random_state=42)

<a class="anchor" id="four-bullet"> 
    
# <span style="color:salmon"> 4. Model Evaluation </span> 
</a>    
</div>
<a href="#top">Top &#129033;</a>

<a class="anchor" id="four-one-bullet"> 
    
## <span style="color:salmon"> 4.1 Training Evaluation </span> 
</a>    
</div>

In [ ]:
# Make training predictions
y_train_pred = model.predict(X_train)

In [ ]:
# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_train, y_train_pred))
print("\nClassification Report:")
print(classification_report(y_train, y_train_pred, target_names=target_names))


Confusion Matrix:
[[  4643   4922     99    339     10      0      0      0]
 [  1180 228911    458   2271    125      0      0      0]
 [    72  28302   4332  20807   1533      0      0      0]
 [    62   5277    183 111387   1736      0      0      0]
 [    11    451    114  17898  20264      0      0      0]
 [     0      4      6   3063    311      0      0      0]
 [     0      1      0     72      5      0      0      0]
 [     0     72     12    286      0      0      0      0]]

Classification Report:
                 precision    recall  f1-score   support

   1. CANCELLED       0.78      0.46      0.58     10013
    2. NON-COMP       0.85      0.98      0.91    232945
    3. MED ONLY       0.83      0.08      0.14     55046
   4. TEMPORARY       0.71      0.94      0.81    118645
5. PPD SCH LOSS       0.84      0.52      0.65     38738
     6. PPD NSL       0.00      0.00      0.00      3384
         7. PTD       0.00      0.00      0.00        78
       8. DEATH       0.00  

c:\Users\abdar\anaconda3\envs\ML_Project\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\abdar\anaconda3\envs\ML_Project\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\abdar\anaconda3\envs\ML_Project\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

<a class="anchor" id="four-two-bullet"> 
    
## <span style="color:salmon"> 4.2 Validation Evaluation </span> 
</a>    
</div>

In [ ]:
# Make validation predictions
y_val_pred = model.predict(X_val)


In [ ]:
# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred, target_names=target_names))


Confusion Matrix:
[[  949  1382    33    96     2     0     0     0]
 [  469 56701   189   725    49     0     0     0]
 [   19  7233   585  5622   401     0     0     0]
 [   18  1603   123 27278   840     0     0     0]
 [    4   124    25  5476  3913     0     0     0]
 [    0     1     1   743    82     0     0     0]
 [    0     0     0    16     3     0     0     0]
 [    0    19     0    81     0     0     0     0]]

Classification Report:
                 precision    recall  f1-score   support

   1. CANCELLED       0.65      0.39      0.48      2462
    2. NON-COMP       0.85      0.98      0.91     58133
    3. MED ONLY       0.61      0.04      0.08     13860
   4. TEMPORARY       0.68      0.91      0.78     29862
5. PPD SCH LOSS       0.74      0.41      0.53      9542
     6. PPD NSL       0.00      0.00      0.00       827
         7. PTD       0.00      0.00      0.00        19
       8. DEATH       0.00      0.00      0.00       100

       accuracy                   

c:\Users\abdar\anaconda3\envs\ML_Project\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\abdar\anaconda3\envs\ML_Project\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\abdar\anaconda3\envs\ML_Project\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

<a class="anchor" id="five-bullet"> 
    
# <span style="color:salmon"> 5. Test Predictions </span> 
</a>    
</div>

<a href="#top">Top &#129033;</a>

In [ ]:
# Make validation predictions
y_test_pred = model.predict(X_test)

In [ ]:
test_id = test_df.index

In [ ]:
submission_df = pd.DataFrame({
    'Claim Identifier': test_id,
    'Claim Injury Type': y_test_pred
})

In [ ]:
# Save the DataFrame to a CSV file
submission_df.to_csv('./submissions/Group49_Version04.csv', index=False)